In [1]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import matplotlib.pyplot as plt
import math
from collections import deque
from pprint import pprint
from sklearn import preprocessing

In [4]:
lenses = arff.loadarff("lenses.arff")
lenses_df = pd.DataFrame(lenses[0])

str_df = lenses_df.select_dtypes([np.object])
str_df = str_df.stack().str.decode('utf-8').unstack()
for col in str_df:
    lenses_df[col]=str_df[col]
dataset=lenses_df

In [92]:
class DT:
    
    def __init__(self, data):
        self.data = data

    def get_entropy(self,col):
        n, counts = np.unique(col, return_counts=True)
#         info_s = 0
#         for i in range(len(n)):
#             info_s += (-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts))
        entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(n))])
        return entropy


    def get_gain(self,data, col_name, target_name):
        info_s = self.get_entropy(data[target_name])
        n, counts= np.unique(data[col_name],return_counts=True)

        info_gain = np.sum([(counts[i]/np.sum(counts))*self.get_entropy(data.where(data[col_name]==n[i]).dropna()[target_name]) for i in range(len(n))])


        #Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])


        return info_s - info_gain

    def ID3(self,data,originaldata,features,target_attribute_name="type",parent_node_class = None):

        if len(features) == 0:
            return parent_node_class
        n = np.unique(data[target_attribute_name])
        if len(n) <= 1:
            
            return n[0]

        #If the dataset is empty, return the mode target feature value in the original dataset
        if len(data)==0:
            return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]

        else:

            target_counts = np.unique(data[target_attribute_name], return_counts=True)[1]
            max_target_count = np.argmax(target_counts)
            parent_node_class = np.unique(data[target_attribute_name])[max_target_count]


            #Select the feature which best splits the dataset
    #         info_gains = []
    #         for i in features:
    #             print("i = ", i, " ", target_attribute_name)
    #             info_gains.append(get_gain(data, i, target_attribute_name))
    #         info_gains = np.array(info_gains)
    #         best_arg = np.argmax(info_gains)
    #         best_feature = features[best_arg]



            item_values = [self.get_gain(data,feature,target_attribute_name) for feature in features] #Return the information gain values for the features in the dataset
            best_feature_index = np.argmax(item_values)
            best_feature = features[best_feature_index]
            print(item_values[best_feature_index])
            #Create the tree structure. The root gets the name of the feature (best_feature) with the maximum information
            #gain in the first run
            tree = {best_feature:{}}


            #Remove the feature with the best inforamtion gain from the feature space
            features = [i for i in features if i != best_feature]
            #Grow a branch under the root node for each possible value of the root node feature

            for value in np.unique(data[best_feature]):
                value = value
                #Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets
                sub_data = data.where(data[best_feature] == value).dropna()

                #Call the ID3 algorithm for each of those sub_datasets with the new parameters --> Here the recursion comes in!
                subtree = self.ID3(sub_data,dataset,features,target_attribute_name,parent_node_class)

                #Add the sub tree, grown from the sub_dataset to the tree under the root node
                tree[best_feature][value] = subtree

            return tree    






    def predict(self,query,tree,default = 1):
        #1.
        for key in list(query.keys()):
            if key in list(tree.keys()):
                #2.
                try:
                    result = tree[key][query[key]] 
                except:
                    return default

                #3.
                result = tree[key][query[key]]
                #4.
                if isinstance(result,dict):
                    return self.predict(query,result)

                else:
                    return result
                
                
                
    def test(self,data,tree):
        #Create new query instances by simply removing the target feature column from the original dataset and 
        #convert it to a dictionary
        queries = data.iloc[:,:-1].to_dict(orient = "records")
        
        print(queries[0])

        #Create a empty DataFrame in whose columns the prediction of the tree are stored
        predicted = pd.DataFrame(columns=["predicted"]) 

        #Calculate the prediction accuracy
        for i in range(len(data)):
            predicted.loc[i,"predicted"] = self.predict(queries[i],tree,1.0) 
        print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data["type"])/len(data))*100,'%')

In [89]:
from scipy.io import arff
import pandas as pd

# Load debug data
lenses = arff.loadarff("lenses.arff")
lenses_df = pd.DataFrame(lenses[0])

str_df = lenses_df.select_dtypes([np.object])
str_df = str_df.stack().str.decode('utf-8').unstack()
for col in str_df:
    lenses_df[col]=str_df[col]
    
lenses_test = arff.loadarff("lenses_test.arff")
lenses_test_df = pd.DataFrame(lenses_test[0])
str_df = lenses_test_df.select_dtypes([np.object])
str_df = str_df.stack().str.decode('utf-8').unstack()
for col in str_df:
    lenses_test_df[col]=str_df[col]    
lenses_np = np.array(lenses_df)
X = lenses_np[:,:-1]
y = lenses_np[:,-1]
f = list(lenses_df.keys())[:4]
dt = DT(lenses_df)
tree = dt.ID3(lenses_df, lenses_df, lenses_df.columns[:-1])

KeyError: 'type'

In [60]:
dt.test(lenses_test_df, tree)

{'age': 'young', 'spectacle_prescrip': 'myope', 'astigmatism': 'no', 'tear_prod_rate': 'reduced'}
The prediction accuracy is:  33.33333333333333 %


In [84]:
tree['tear_prod_rate']

{'normal': {'astigmatism': {'no': {'age': {'pre_presbyopic': 'soft',
     'presbyopic': {'spectacle_prescrip': {'hypermetrope': 'none',
       'myope': 'none'}},
     'young': 'soft'}},
   'yes': {'spectacle_prescrip': {'hypermetrope': {'age': {'pre_presbyopic': 'none',
       'presbyopic': 'none',
       'young': 'none'}},
     'myope': 'hard'}}}},
 'reduced': 'none'}

In [83]:
tree['tear_prod_rate']['reduced']

'none'

In [93]:
zoo = arff.loadarff("zoo.arff")
zoo_df = pd.DataFrame(zoo[0])

str_df = zoo_df.select_dtypes([np.object])
str_df = str_df.stack().str.decode('utf-8').unstack()
for col in str_df:
    zoo_df[col]=str_df[col]
zoo_np = np.array(zoo_df)
X = zoo_np[:,-1]
y = zoo_np[:-1]

# Train Decision Tree

# Load evaluation test data
zoo_test = arff.loadarff("zoo_test.arff")
zoo_test_df = pd.DataFrame(zoo_test[0])

str_df = zoo_test_df.select_dtypes([np.object])
str_df = str_df.stack().str.decode('utf-8').unstack()
for col in str_df:
    zoo_test_df[col]=str_df[col]
zoo_test_np = np.array(zoo_test_df)
X_test = zoo_test_np[:,:-1]
y_test = zoo_test_np[:,-1]

dt = DT(zoo_df)
tree = dt.ID3(zoo_df, zoo_df, zoo_df.columns[:-1])
dt.test(zoo_test_df, tree)

1.3630469031539394
0.8865408928220899
0.9852281360342515
0.6962122601251458
0.8256265261578954
0.6892019851173654
0.8631205685666308
0.7219280948873623
0.7219280948873623
{'hair': 'F', 'feathers': 'F', 'eggs': 'T', 'milk': 'T', 'airborne': 'T', 'predator': 'F', 'aquatic': 'F', 'toothed': 'F', 'backbone': 'T', 'breathes': 'T', 'venomous': 'F', 'fins': 'T', 'legs': '8', 'tails': 'F', 'domestic': 'F', 'catsize': 'T'}
The prediction accuracy is:  14.7 %


In [34]:
labels = list(lenses_df.keys())
print((-15/24)*np.log2(15/24)+(-5/24)*np.log2(5/24)+(-4/24)*np.log2(4/24))

1.3260875253642983


In [26]:
dt = CustomDecisionTree()

In [27]:
lenses_df.replace('young',0,inplace=True)
lenses_df.replace('pre_presbyopic',1,inplace=True)
lenses_df.replace('presbyopic',2,inplace=True)

In [28]:
lenses_df.replace('myope',0,inplace=True)
lenses_df.replace('hypermetrope',1,inplace=True)

In [29]:
lenses_df.replace('no',0,inplace=True)
lenses_df.replace('yes',1,inplace=True)

In [30]:
lenses_df.replace('reduced',0,inplace=True)
lenses_df.replace('normal',1,inplace=True)

In [31]:
lenses_df.replace('none',0,inplace=True)
lenses_df.replace('soft',1,inplace=True)
lenses_df.replace('hard',2,inplace=True)

In [32]:
d = lenses_df.astype(float).values.tolist()
print(dt.createTree(d, labels))

-0.3679197916065703 -1
-0.21541354172676297 -0.3679197916065703
-0.21541354172676297 -0.21541354172676297
the best feature to split is spectacle_prescrip -0.21541354172676297
-0.4025062498798073 -1
-0.2641604167868593 -0.4025062498798073
the best feature to split is astigmatism -0.2641604167868593
-0.138345833092948 -1
the best feature to split is age -0.138345833092948
the best feature to split is tear_prod_rate -1
the best feature to split is tear_prod_rate -1
-0.138345833092948 -1
the best feature to split is age -0.138345833092948
the best feature to split is tear_prod_rate -1
the best feature to split is tear_prod_rate -1
the best feature to split is tear_prod_rate -1
-0.4025062498798073 -1
-0.2641604167868593 -0.4025062498798073
the best feature to split is astigmatism -0.2641604167868593
-0.138345833092948 -1
the best feature to split is age -0.138345833092948
the best feature to split is tear_prod_rate -1
the best feature to split is tear_prod_rate -1
the best feature to split 

In [2]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [3]:
le.fit([1,2,2,6])

LabelEncoder()

In [5]:
le.classes_

array([1, 2, 6])

In [6]:
le.transform([1,2,2,6])

array([0, 1, 1, 2], dtype=int64)